In [78]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import pybaseball
pd.options.mode.chained_assignment = None 
#https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

This block will download all of the raw data from 2017 through 2021


Some thoughts:
    I have two options:
        1) Download the data and cut it down/clean it to what it I want,
        2) Download the data, save that, then clean it and save that
        
        Option 1 is more memory efficient but option 1 is more time but 
        option 2 is more time efficient if/when I make mistakes
        
        So for now I'm going with option two and if that is super fast then I will
        delete that result and combine them into one script. Okay here goes

In [11]:
def get_data_from_mlb(start_year, end_year = None):
    
    if end_year == None:
        end_year = start_year + 1
    
    dfs = [] #https://github.com/jldbc/pybaseball/blob/master/EXAMPLES/imputed_derivation.ipynb
    for year in range(start_year, end_year + 1):
        print(f"Starting year {year}")
        dfs.append(pybaseball.statcast(start_dt=f'{year}-03-21', end_dt=f'{year}-12-01',verbose=False))
        #I'm overshooting the start and end here to make sure I don't miss anything
        #The statcast function is written in parallel, so that helps speed up this process
        
    # https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
    print("Info succesfully downloaded from statcast")
    return pd.concat(dfs) 
    

In [13]:
if __name__ == '__main__':

    result = get_data_from_mlb(2017, 2021)
    
    folder_name = "data_files/"
    
    print("Writing raw data to csv")
    result.to_csv(folder_name + "raw_data_from_statcast.csv", index = False)
    
    
    
    key_points = result[[ 
                            "game_date",
                            "description",
                            "home_team", #Team doesn't matter
                            "away_team", #Remove if run again
                            "game_year",
                            "plate_x",
                            "plate_z",
                            "sz_top",
                            "sz_bot",
                            "game_pk"
    ]]
    
    print("Writing key_points data to csv")
    key_points.to_csv(folder_name + "key_points.csv", index = False)
    
    

Starting year 2017


100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [00:33<00:00,  6.46it/s]


Starting year 2018


100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:04<00:00,  3.32it/s]


Starting year 2019


...

Starting year 2020


100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [00:27<00:00,  3.57it/s]


Starting year 2021


100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [01:05<00:00,  3.64it/s]


Info succesfully downloaded from statcast
Writing raw data to csv
Writing key_points data to csv


key_points.csv has all of the pitch information I need to work with. 

Now I need to group them by game, and rate the ump's performance for that game

In [116]:
class Rate_Game:
    """
    I verified that these methods work using game_pk:490098
    I had it print the results here and then wrote it as a 
    CSV and compared in excel
    """
    
    """
    Inputs:
        game_df: 
            pandas df of all of the pitches 
            In most cases this will be one game, but I guess I could use this to get overall averages too
        game_pk:
            ID of the game
    Output:
        A pandas series that will have the following information:
        game_pk, game_date, ump accuracy, ump consistency
    """
    def grade_game(game_df, game_pk):
        
        game_df = game_df[
            np.where(  
            (game_df['description'] == 'ball') | \
            (game_df['description'] == 'blocked_ball') |  \
            (game_df['description'] == 'called_strike'),
             True, False   
            )] 
        #Only looking at calls the umpire made
        
        
        return pd.Series({
            'game_pk' : game_pk,
            'game_date' : game_df['game_date'].iloc[0], #Date pulled from first row
            'game_year' : game_df['game_year'].iloc[0], #Date pulled from first row
            'umpire_accuracy' : Rate_Game.grade_accuracy(game_df),
            'umpire_consistency' : Rate_Game.grade_consistency(game_df),
        })
        
   
    """ #TODO
    Inputs:
        game_df: 
            pandas df of all of the pitches 
    Output:
        Score of how consistent the ump was
    """   
    def grade_consistency(game_df):
        return None
    
        
    """
    Inputs:
        game_df: 
            pandas df of all of the pitches 
    Output:
        How many pitches were called correctly in the zone divided by total number of called pitches
    """   
    def grade_accuracy(game_df):
        half_width_ball = 2.9/12
        sz_L = (-8.5/12) - half_width_ball
        sz_R = (8.5/12) + half_width_ball
        
        #https://stackoverflow.com/questions/30631841/pandas-how-do-i-assign-values-based-on-multiple-conditions-for-existing-columns
        game_df["in_zone"] = np.where(
            (game_df["plate_x"] > sz_L) & \
            (game_df["plate_x"] < sz_R) & \
            (game_df["plate_z"] > (game_df["sz_bot"] - half_width_ball)) & \
            (game_df["plate_z"] < (game_df["sz_top"] + half_width_ball)),
            True, False
        )
        
        game_df["correct_call"] = \
            ((game_df["in_zone"] == True) & (game_df['description'] == 'called_strike')) |\
              ((game_df["in_zone"] == False) & (game_df['description'] == 'blocked_ball')) |\
              ((game_df["in_zone"] == False) & (game_df['description'] == 'ball'))

        return game_df[game_df["correct_call"] == True].shape[0] /game_df.shape[0]
    
    


In [10]:
key_points = pd.read_csv("data_files/key_points.csv")
game_groups = key_points.groupby(['game_pk'])

In [115]:
""" #Parallelized in next block
game_groups = key_points.groupby(['game_pk'])
for game_pk, game_df in game_groups.__iter__(): #iters through every game
    print(Rate_Game.grade_game(game_df, game_pk), "\n")
    #break
"""

' #Parallelized in next block\ngame_groups = key_points.groupby([\'game_pk\'])\nfor game_pk, game_df in game_groups.__iter__(): #iters through every game\n    print(Rate_Game.grade_game(game_df, game_pk), "\n")\n    #break\n'

In [126]:
game_groups = key_points.groupby(['game_pk'])
series_list = Parallel(n_jobs = -1, batch_size = 5)(
    delayed(Rate_Game.grade_game)(game_df, game_pk) for game_pk, game_df in game_groups.__iter__())
pd.concat(series_list, axis = 1).T


,game_pk,game_date,game_year,umpire_accuracy,umpire_consistency
0,490098,2017-04-03,2017,0.847134,None
1,490099,2017-04-02,2017,0.865922,None
2,490100,2017-04-03,2017,0.853933,None
3,490101,2017-04-03,2017,0.865385,None
4,490102,2017-04-04,2017,0.909639,None
...,...,...,...,...,...
11027,660933,2021-10-08,2021,0.900901,None
11028,660934,2021-10-08,2021,0.924138,None
11029,660936,2021-10-07,2021,0.915152,None
11030,660937,2021-10-06,2021,0.915663,None
